# Random Forest

In [38]:
from pyspark.sql.types import BooleanType
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import expr
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from helpers.helper_functions import translate_to_file_string

import pandas as pd
from IPython.display import display, HTML
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix

#Change Column Names
def delete_space(df):
    names = df.schema.names
    for name in names:
        newName = name.replace(" ","")
        df = df.withColumnRenamed(name, newName)
    return df

In [39]:
inputFile = translate_to_file_string("../data/data.csv")

Spark session creation 

In [40]:
spark = (SparkSession
       .builder
       .appName("Modell_randomForest")
       .getOrCreate())

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:37459)
Traceback (most recent call last):
  File "/home/ubuntu/spark-3.0.1-bin-hadoop2.7/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 977, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ubuntu/spark-3.0.1-bin-hadoop2.7/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1115, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused


Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:37459)

DataFrame creation using an ifered Schema 

In [20]:
# create a DataFrame using an ifered Schema 
df = spark.read.option("header", "true") \
       .option("inferSchema", "true") \
       .option("delimiter", ";") \
       .csv(inputFile) 

Prepare training and test data.

In [22]:
df = delete_space(df)
df.printSchema()

#Create Pandas DataFrame
df_pandas = df.toPandas()
pandasCol = list(df_pandas)
for col in pandasCol:
    if df_pandas[col].dtypes=='object':
        #ToInteger
        df_pandas[col]= pd.factorize(df_pandas[col])[0]



df_pandas.info()

HTML(df_pandas.head(5).to_html())

df2 = spark.createDataFrame(df_pandas)

featureCols = df2.columns.copy()
featureCols.remove("CustomerID")
featureCols.remove("Contract")
print(featureCols)

assembler =  VectorAssembler(outputCol="features", inputCols=featureCols)

#Keep Nullvalues 
assembler.setHandleInvalid("keep")

root
 |-- CustomerID: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- SeniorCitizen: integer (nullable = true)
 |-- Partner: string (nullable = true)
 |-- Dependents: string (nullable = true)
 |-- Tenure: integer (nullable = true)
 |-- PhoneService: string (nullable = true)
 |-- MultipleLines: string (nullable = true)
 |-- InternetService: string (nullable = true)
 |-- OnlineSecurity: string (nullable = true)
 |-- OnlineBackup: string (nullable = true)
 |-- DeviceProtection: string (nullable = true)
 |-- TechSupport: string (nullable = true)
 |-- StreamingTV: string (nullable = true)
 |-- StreamingMovies: string (nullable = true)
 |-- Contract: string (nullable = true)
 |-- PaperlessBilling: string (nullable = true)
 |-- PaymentMethod: string (nullable = true)
 |-- MonthlyCharges: double (nullable = true)
 |-- TotalCharges: double (nullable = true)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6499 entries, 0 to 6498
Data columns (total 20 columns):
 #   Column

VectorAssembler_3f83f0f4424f

In [24]:
labledPointDataSet = assembler.transform(df2)
splits = labledPointDataSet.randomSplit([0.9, 0.1 ], 12345)
training = splits[0]
test = splits[1]

Random Forest Classifier

In [26]:
#Stanard Ergebnis: Test Error ~0.175
# rf = RandomForestClassifier(labelCol="Contract", featuresCol="features")

# verbessertes Ergebnis: Test Error ~0.098
rf = RandomForestClassifier(labelCol="Contract", featuresCol="features" ,impurity="gini", \
                 minInstancesPerNode=1350, featureSubsetStrategy='sqrt', subsamplingRate=1, seed= 12345, numTrees=1000)

Train the model 

In [28]:
rfModel = rf.fit(training)

Test the model

In [ ]:
predictions = rfModel.transform(test)
predictions.show()

In [29]:
evaluator = BinaryClassificationEvaluator(labelCol="Contract",rawPredictionCol="rawPrediction", metricName="areaUnderROC")
accuracy = evaluator.evaluate(predictions)
print("Test Error",(1.0 - accuracy))

Py4JJavaError: An error occurred while calling o718.evaluate.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 35.0 failed 1 times, most recent failure: Lost task 2.0 in stage 35.0 (TID 205, 10.0.2.15, executor driver): java.lang.OutOfMemoryError: Java heap space
	at java.io.ObjectInputStream$HandleTable$HandleList.add(ObjectInputStream.java:3985)
	at java.io.ObjectInputStream$HandleTable.markDependency(ObjectInputStream.java:3804)
	at java.io.ObjectInputStream.readObject(ObjectInputStream.java:504)
	at java.io.ObjectInputStream.readObject(ObjectInputStream.java:461)
	at scala.collection.mutable.HashMap.$anonfun$readObject$1(HashMap.scala:193)
	at scala.collection.mutable.HashMap$$Lambda$2077/924590470.apply(Unknown Source)
	at scala.collection.mutable.HashTable.init(HashTable.scala:110)
	at scala.collection.mutable.HashTable.init$(HashTable.scala:89)
	at scala.collection.mutable.HashMap.init(HashMap.scala:44)
	at scala.collection.mutable.HashMap.rea

In [31]:
spark.stop()

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:37459)
Traceback (most recent call last):
  File "/home/ubuntu/spark-3.0.1-bin-hadoop2.7/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 977, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ubuntu/spark-3.0.1-bin-hadoop2.7/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1115, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:37459)
Traceback (most recent call last):
  File "/home/ubuntu/spark-3.0.1-bin-hadoop2.7/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 977, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empt

Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:37459)